In [5]:
import agama
import gizmo_analysis as gizmo
import h5py
import numpy as np
import utilities as ut
from gc_utils import iteration_name, open_snapshot, snapshot_name  # type: ignore

In [6]:
simulation = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_kinematics/data/"

fire_dir = sim_dir + simulation + "/" + simulation + "_res7100/"

In [7]:
it_lst = np.array([1])
snapshot = 600

In [8]:
part = gizmo.io.Read.read_snapshots("all", "index", snapshot, fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 57060074 particles
    star      (id = 4): 13976485 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'dark2', 'gas', 'star']

* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.Partic

In [10]:
sim = simulation

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
potential_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snapshot, snapshot)

proc_data = h5py.File(proc_file, "a")  # open processed data file
pot_nbody = agama.Potential(potential_file)
af = agama.ActionFinder(pot_nbody, interp=False)

In [56]:
snap_id = snapshot_name(snapshot)
host_index = 0

for it in it_lst:
    it_id = iteration_name(it)

    gc_id_snap = proc_data[it_id]["snapshots"][snap_id]["gc_id"]

    if len(gc_id_snap) is None:
        continue

    ptype_byte_snap = proc_data[it_id]["snapshots"][snap_id]["ptype"]
    ptype_snap = [ptype.decode("utf-8") for ptype in ptype_byte_snap]

    host_name = ut.catalog.get_host_name(host_index)

    x_lst = []
    y_lst = []
    z_lst = []
    vx_lst = []
    vy_lst = []
    vz_lst = []

    r_cyl_lst = []
    phi_cyl_lst = []
    vr_cyl_lst = []
    vphi_cyl_lst = []

    init_cond_lst = []

    r_lst = []
    # r_per_lst = []
    # r_apo_lst = []

    ep_fire_lst = []
    # ep_agama_lst = []
    ek_lst = []
    # et_lst = []

    lx_lst = []
    ly_lst = []
    lz_lst = []

    # jr_lst = []
    # jz_lst = []
    # jphi_lst = []

    for gc, ptype in zip(gc_id_snap, ptype_snap):
        idx = np.where(part[ptype]["id"] == gc)[0][0]
        pos_xyz = part[ptype].prop(f"{host_name}.distance.principal", idx)
        vel_xyz = part[ptype].prop(f"{host_name}.velocity.principal", idx)

        pos_cyl = part[ptype].prop(f"{host_name}.distance.principal.cylindrical", idx)
        vel_cyl = part[ptype].prop(f"{host_name}.velocity.principal.cylindrical", idx)

        ep_fir = part[ptype]["potential"][idx]

        init_cond = np.concatenate((pos_xyz, vel_xyz))

        x, y, z = pos_xyz
        vx, vy, vz = vel_xyz

        r_cyl, phi_cyl, _ = pos_cyl
        vr_cyl, vphi_cyl, _ = vel_cyl

        ek = 0.5 * np.linalg.norm(vel_xyz) ** 2

        r = np.linalg.norm(pos_xyz)

        lx = y * vz - z * vy
        ly = z * vx - x * vz
        lz = x * vy - y * vx

        x_lst.append(x)
        y_lst.append(y)
        z_lst.append(z)
        vx_lst.append(vx)
        vy_lst.append(vy)
        vz_lst.append(vz)

        r_cyl_lst.append(r_cyl)
        phi_cyl_lst.append(phi_cyl)
        vr_cyl_lst.append(vr_cyl)
        vphi_cyl_lst.append(vphi_cyl)

        r_lst.append(r)

        ep_fire_lst.append(ep_fir)
        ek_lst.append(ek)

        lx_lst.append(lx)
        ly_lst.append(ly)
        lz_lst.append(lz)

        init_cond_lst.append(init_cond)

ioms = af(init_cond_lst)
jr_lst = ioms[:, 0]
jz_lst = ioms[:, 1]
jphi_lst = ioms[:, 2]

radii_pos = pot_nbody.Rperiapo(init_cond_lst)
r_per = radii_pos[:, 0]
r_apo = radii_pos[:, 1]

ep_aga = pot_nbody.potential(np.array(init_cond_lst)[:, 0:3])
et = ep_aga + ek_lst

In [59]:
lz_lst[0], jphi_lst[0]

(-817.0752218122228, -817.0752218122228)